# Getting started with the EB-NeRD

#### Check unique topics for articles (and difference between topics in small and large dataset)

In [2]:
from tqdm import tqdm
import pandas as pd


articles_small = pd.read_parquet("data/eb-nerd/ebnerd_small/articles.parquet")
articles_large = pd.read_parquet("data/eb-nerd/ebnerd_large/articles.parquet")


exploded_df = articles_small.explode('topics')
unique_tags = exploded_df['topics'].unique()
unique_tags_list_small = unique_tags.tolist()


exploded_df = articles_large.explode('topics')
unique_tags = exploded_df['topics'].unique()
unique_tags_list_large = unique_tags.tolist()


set1 = set(unique_tags_list_small)
set2 = set(unique_tags_list_large)
diff1 = list(set1 - set2)
diff2 = list(set2 - set1)
print(diff1)

[nan]


#### aa

In [5]:
unique_categories_small = articles_small['category_str'].unique()
print("Unique categories small:", unique_categories_small)

unique_categories_large = articles_large['category_str'].unique()
print("Unique categories large:", unique_categories_large)

set1 = set(unique_categories_small)
set2 = set(unique_categories_large)
diff1 = list(set1 - set2)
diff2 = list(set2 - set1)

print('Categories in small but not in large ', diff1)
print('Categories in large but not in small ', diff2)

Unique categories small: ['krimi' 'underholdning' 'sport' 'nyheder' 'sex_og_samliv' 'incoming'
 'ferie' 'musik' 'side9' 'nationen' 'forbrug' 'biler' 'haandvaerkeren'
 'om_ekstra_bladet' 'plus' 'services' 'horoskoper' 'vin' 'video'
 'opinionen' 'bibliotek' 'dagsorden' 'play' 'podcast' 'auto' 'penge']
Unique categories large: ['underholdning' 'nyheder' 'sport' 'musik' 'krimi' 'sex_og_samliv' 'ferie'
 'biler' 'incoming' 'forbrug' 'webtv' 'bibliotek' 'side9' 'haandvaerkeren'
 'plus' 'nationen' 'vin' 'om_ekstra_bladet' 'abonnement' 'services'
 'video' 'horoskoper' 'opinionen' 'tilavis' 'eblive' 'dagsorden'
 'webmaster-test-sektion' 'migration_catalog' 'podcast' 'play' 'rssfeed'
 'auto' 'penge']
Categories in small but not in large  []
Categories in large but not in small  ['eblive', 'abonnement', 'webmaster-test-sektion', 'webtv', 'tilavis', 'migration_catalog', 'rssfeed']


In [20]:
import pandas as pd

df1 = pd.read_parquet("ebnerd-benchmark\data\ebnerd_demo/preprocessed.parquet")
df2 = pd.read_parquet("ebnerd-benchmark\data\ebnerd_demo/preprocessed_and_title_enhanced.parquet")

print(len(df1))
print(len(df2))

# # Find IDs in df1 not in df2
# unique_in_df1 = df1[~df1['article_id'].isin(df2['article_id'])]

# # Find IDs in df2 not in df1
# unique_in_df2 = df2[~df2['article_id'].isin(df1['article_id'])]

# print("Unique in df1:")
# print(len(unique_in_df1))

# print("Unique in df2:")
# print(len(unique_in_df2))

11777
11176


In [22]:
# Merge the dataframes on 'article_id' with a left join
merged_df = pd.merge(df1, df2[['article_id', 'title']], on='article_id', how='left', suffixes=('', '_enhanced'))

# Replace the title from df1 with the title from df2 if it exists
merged_df['title'] = merged_df['title_enhanced'].where(merged_df['title_enhanced'].notna(), merged_df['title'])

# Drop the enhanced title column as it's no longer needed
merged_df.drop(columns=['title_enhanced'], inplace=True)

merged_df.to_parquet("ebnerd-benchmark\data\ebnerd_demo/preprocessed_and_title_enhanced_fixed.parquet")

## Images/No images statistics

In [38]:
import pandas as pd

# Load data
articles = pd.read_parquet("ebnerd-benchmark/data/ebnerd_small/preprocessed_and_images.parquet")
behaviors = pd.read_parquet("ebnerd-benchmark/data/ebnerd_small/train/behaviors.parquet")

# Preprocess 'article_id' column in articles for faster access
article_id_set = set(articles['article_id'])

# Calculate images shown and not shown
images_shown = 0
no_images_shown = 0

# Explode the article_ids_inview to have one per row for easy join
behaviors_exploded = behaviors.explode('article_ids_inview')

# Join behaviors with articles on article_id
merged = behaviors_exploded.join(articles.set_index('article_id'), on='article_ids_inview')

# Calculate total
total = len(merged)

# Vectorized operation to check for image captions
merged['has_image'] = merged['image_caption_text'].apply(lambda x: len(str(x)) > 0)

# Summarize the results
images_shown = merged['has_image'].sum()
no_images_shown = total - images_shown

print('Articles with images shown: ', images_shown)
print('Articles without images shown: ', no_images_shown)

Articles with images shown:  2358623
Articles without images shown:  227124


In [40]:
import pandas as pd

# Load data
articles = pd.read_parquet("ebnerd-benchmark/data/ebnerd_small/preprocessed_and_images.parquet")
behaviors = pd.read_parquet("ebnerd-benchmark/data/ebnerd_small/train/behaviors.parquet")

# Preprocess 'article_id' column in articles for faster access
article_id_set = set(articles['article_id'])

# Calculate images shown and not shown
images_clicked = 0
no_images_clicked = 0

# Explode the article_ids_inview to have one per row for easy join
behaviors_exploded = behaviors.explode('article_ids_clicked')

# Join behaviors with articles on article_id
merged = behaviors_exploded.join(articles.set_index('article_id'), on='article_ids_clicked')

# Calculate total
total = len(merged)

# Vectorized operation to check for image captions
merged['has_image'] = merged['image_caption_text'].apply(lambda x: len(str(x)) > 0)

# Summarize the results
images_clicked = merged['has_image'].sum()
no_images_clicked = total - images_clicked

print('Articles with images clicked: ', images_clicked)
print('Articles without images clicked: ', no_images_clicked)

Articles with images clicked:  212877
Articles without images clicked:  21400


In [42]:
prob_with_images = images_clicked / images_shown
prob_without_images = no_images_clicked / no_images_shown


print(f'Probability of clicking on an article with an image: {prob_with_images:.2%}')
print(f'Probability of clicking on an article without an image: {prob_without_images:.2%}')

Probability of clicking on an article with an image: 9.03%
Probability of clicking on an article without an image: 9.42%
